In [ ]:
# Tomado de: https://www.analyticsvidhya.com/blog/2020/10/multivariate-multi-step-time-series-forecasting-using-stacked-lstm-sequence-to-sequence-autoencoder-in-tensorflow-2-0-keras/
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv('/content/gdrive/Shareddrives/Network Science/PROYECTO/Serie de tiempo/serie.csv', sep=',')
data

,Source,Target,Time,Distrito
0,254,0,0,3
1,254,5297,0,3
2,254,5504,0,3
3,254,6525,2,9
4,254,22048,2,9
...,...,...,...,...
148647,33655,25905,587,3
148648,65324,2693,587,3
148649,66044,2932,587,6
148650,195402,188506,587,3


In [ ]:
data.head()

,Source,Target,Time,Distrito
0,254,0,0,3
1,254,5297,0,3
2,254,5504,0,3
3,254,6525,2,9
4,254,22048,2,9


In [ ]:
data['Distrito'] = pd.to_numeric(data['Distrito'])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148652 entries, 0 to 148651
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   Source    148652 non-null  int64
 1   Target    148652 non-null  int64
 2   Time      148652 non-null  int64
 3   Distrito  148652 non-null  int64
dtypes: int64(4)
memory usage: 4.5 MB


In [ ]:
# Conversion de los datos a numpy array
valores = data.values

# Construcion de escalador
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(valores)
print(scaler)

# Escalamiento de los valores
normalizados = scaler.transform(valores)
df_normalizados = pd.DataFrame(normalizados,
                              index=data.index,
                              columns=data.columns)
df_normalizados

MinMaxScaler()


,Source,Target,Time,Distrito
0,0.001295,0.000000,0.000000,0.2
1,0.001295,0.027009,0.000000,0.2
2,0.001295,0.028064,0.000000,0.2
3,0.001295,0.033270,0.003407,0.8
4,0.001295,0.112419,0.003407,0.8
...,...,...,...,...
148647,0.171651,0.132085,1.000000,0.2
148648,0.333174,0.013731,1.000000,0.2
148649,0.336846,0.014950,1.000000,0.5
148650,0.996613,0.961162,1.000000,0.2


In [ ]:
data.info()
y = np.array(df_normalizados['Distrito'])
X = np.array(df_normalizados[['Source','Target','Time']])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148652 entries, 0 to 148651
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   Source    148652 non-null  int64
 1   Target    148652 non-null  int64
 2   Time      148652 non-null  int64
 3   Distrito  148652 non-null  int64
dtypes: int64(4)
memory usage: 4.5 MB


In [ ]:
df_normalizados.head()

,Source,Target,Time,Distrito
0,0.001295,0.000000,0.000000,0.2
1,0.001295,0.027009,0.000000,0.2
2,0.001295,0.028064,0.000000,0.2
3,0.001295,0.033270,0.003407,0.8
4,0.001295,0.112419,0.003407,0.8


In [ ]:
df = df_normalizados.replace('?', np.nan)
df.isnull().sum()

Source      0
Target      0
Time        0
Distrito    0
dtype: int64

In [ ]:
def fill_missing(values):
    one_day = 60*24
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(values[row][col]):
                values[row,col] = values[row-one_day,col]
df = df
fill_missing(data.values)
df.isnull().sum()


Source      0
Target      0
Time        0
Distrito    0
dtype: int64

In [ ]:
daily_df = df

In [ ]:
train_number = int (len(df)*0.7)
train_df,test_df = daily_df[0:train_number], daily_df[train_number+1:len(df)] 
train_number

104056

In [ ]:
len(test_df)

44595

In [ ]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

<ipython-input-17-468d57fa9ccd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-17-468d57fa9ccd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
<ipython-input-17-468d57fa9ccd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)


In [ ]:
n_past = 10
n_future = 2
n_features = 4

In [ ]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], n_past, -1))
y_train = y_train.reshape((y_train.shape[0], n_future, -1))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], n_past, -1))
y_test = y_test.reshape((y_test.shape[0], n_future, -1))


In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 4)]      0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 100),        42000       ['input_1[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 repeat_vector (RepeatVector)   (None, 2, 100)       0           ['lstm[0][0]']                   
                                                                                              

In [ ]:
import datetime
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.losses.CategoricalCrossentropy() # tf.keras.losses.Huber()

model_e1d1.compile(
    optimizer=optimizer, 
    loss=loss_fn,
    weighted_metrics=["acc"]
)
history_e1d1 = model_e1d1.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_test,y_test),
    batch_size=32,
    verbose=0,
    callbacks=[EarlyStopping(monitor='acc', patience=20)],
)

In [ ]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e1d1

1394/1394 [==============================] - 4s 3ms/step


array([[[  0.7948893 ,   4.1650095 , -11.230161  ,   3.512796  ],
        [ -3.8835747 ,  -2.6230416 ,  17.526278  ,  -3.034396  ]],

       [[  0.89647144,   4.4344034 , -12.420875  ,   2.9011037 ],
        [ -4.107465  ,  -2.5099816 ,  17.160978  ,  -2.9036112 ]],

       [[  1.2270534 ,   4.6366854 , -11.299916  ,   2.7724211 ],
        [ -3.931786  ,  -2.8011463 ,  17.53912   ,  -2.982059  ]],

       ...,

       [[  0.9151977 ,   4.3801355 , -11.856521  ,   3.0686483 ],
        [ -3.9110897 ,  -2.5140486 ,  17.317282  ,  -2.974539  ]],

       [[  0.8745691 ,   4.2907414 , -11.43083   ,   3.1948733 ],
        [ -3.837154  ,  -2.5324674 ,  17.27403   ,  -3.0058827 ]],

       [[  1.0427351 ,   4.588064  , -11.901315  ,   2.7863688 ],
        [ -3.8882458 ,  -2.462134  ,  16.944399  ,  -3.0169268 ]]],
      dtype=float32)